In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [1]:
from tensorflow.keras.layers import Embedding, Dot, Add, Flatten, Reshape
from tensorflow import keras
import pandas as pd
import numpy as np
import pickle

In [91]:
with open('matrixfactorizing_als_me/urm_sparse.pkl', 'rb') as f:
    train = pickle.load(f)

In [92]:
with open('matrixfactorizing_als_me/urm_sparse_test.pkl', 'rb') as f:
    test = pickle.load(f)

In [77]:
users = train.nonzero()[0]
movies = train.nonzero()[1]

In [82]:
users_test = test.nonzero()[0]
movies_test = test.nonzero()[1]

In [78]:
N = np.unique(users).shape[0] 
M = np.unique(movies).shape[0]

In [79]:
users.shape, movies.shape

((5566601,), (5566601,))

In [90]:
users_test.shape, movies_test.shape

((281497,), (281497,))

In [93]:
mu = 3.383387277083448

train.data -= mu
test.data -= mu

In [94]:
#size latent space
K=10

In [95]:
targets = train.data
vals = test.data

In [96]:
targets.shape, vals.shape

((5566601,), (281497,))

In [98]:
u = keras.Input(shape=(None,))
m = keras.Input(shape=(None,))

In [99]:
u.shape, m.shape

(TensorShape([None, None]), TensorShape([None, None]))

In [100]:
e_u = Embedding(input_dim=N, output_dim=K)(u)
e_m = Embedding(input_dim=M, output_dim=K)(m)

In [101]:
e_u.shape, e_m.shape

(TensorShape([None, None, 10]), TensorShape([None, None, 10]))

In [102]:
bias_u = Embedding(N, 1)(u)
bias_m = Embedding(M, 1)(m)

In [103]:
bias_u.shape, bias_m.shape

(TensorShape([None, None, 1]), TensorShape([None, None, 1]))

In [104]:
#bias_m_resh = Reshape((1, M), input_shape=(M,1))(bias_m)

In [105]:
bias_m_resh.shape

TensorShape([None, 1, 4237])

In [106]:
r = Dot(axes=2)([e_u, e_m])

In [107]:
r.shape

TensorShape([None, None, None])

In [108]:
r = Add()([r, bias_u, bias_m])

In [109]:
r.shape

TensorShape([None, None, None])

In [110]:
model = keras.Model(inputs=[u,m], outputs=r)
model.compile(loss='mse', optimizer='sgd', metrics=['mse'])

In [111]:
users.shape

(5566601,)

In [113]:
loss = model.fit(x=[users, movies], y=targets, epochs=1, validation_data=([users_test, movies_test], vals))
plot(loss)

Train on 5566601 samples, validate on 281497 samples
5566601/5566601 [==============================] - 589s 106us/sample - loss: 0.8816 - mse: 0.8816 - val_loss: 0.7434 - val_mse: 0.7434


NameError: name 'plot' is not defined

In [114]:
targets

array([-0.38338733,  1.6166127 ,  0.6166127 , ..., -1.3833873 ,
       -0.38338733, -1.8833873 ], dtype=float32)

In [115]:
vals

array([-0.38338733,  0.6166127 ,  0.6166127 , ...,  0.6166127 ,
       -0.8833873 ,  0.11661267], dtype=float32)